In [15]:
import requests as rq
import random
import ujson
from bs4 import BeautifulSoup
from tqdm.auto import tqdm
from time import sleep
import pandas as pd

In [5]:
def get_random_waittime():
    waittime = random.weibullvariate(1, 5)/1.5
    return waittime

In [6]:
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/89.0.4389.114 Safari/537.36',
    'Accept-Language': 'en,en-US;q=0,5',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,/;q=0.8'
}

In [7]:
with open('micro_dataset1.json', 'r') as f:
  j = f.read()

jj = ujson.loads(j)

In [10]:
l = []

for e in tqdm(jj[:15]):
    url = e['ARTWORK_URL']
    art_id = e['ART_ID']
    page_html = rq.get(url, headers=headers)
    while page_html.status_code != 200:
        page_html = rq.get(url, headers=headers)
        sleep(get_random_waittime())
    soup = BeautifulSoup(page_html.content)
    j = soup.find('script', {'id': '__NEXT_DATA__'}).contents[0]
    j = ujson.loads(j)
    views = j['props']['pageProps']['initialState']['page']['data']['artwork']['views']
    likes = j['props']['pageProps']['initialState']['page']['data']['artwork']['likes']
    # print(views)
    l.append({'art_id': art_id, 'artwork_url': url, 'views': views, 'likes': likes})
    sleep(get_random_waittime()/4)


  0%|          | 0/15 [00:00<?, ?it/s]

In [16]:
df = pd.DataFrame.from_records(l)
df.head()

,art_id,artwork_url,views,likes
0,893689,https://www.saatchiart.com/art/Painting-untitl...,1164,16
1,877165,https://www.saatchiart.com/art/Painting-Cai-Gu...,578,4
2,854554,https://www.saatchiart.com/art/Painting-La-Sos...,435,0
3,846988,https://www.saatchiart.com/art/Painting-untitl...,1556,11
4,846977,https://www.saatchiart.com/art/Painting-grip/7...,1253,8


In [17]:
df.to_csv('d.csv', index=False)